managed tables & unmanaged(external) tables

In [0]:
from delta.tables import DeltaTable

In [0]:
data = [(1,"Sana", 5000),
        (2,"Priya", 6000),
        (3,"rekha", 7000),
        (4,"ravi", 8000)]
columns = ["id","name","salary"]
df = spark.createDataFrame(data,columns)
df.show()

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("employee_managed")

In [0]:
spark.sql("DESCRIBE DETAIL employee_managed").display()

In [0]:
%sql
select * from employee_managed;

In [0]:
df2 =  spark.sql("select * from employee_managed")
df2.show()

external delta tables

In [0]:
external_path = "/mnt/inputdata/delta/employee_external"
df.write.format("delta").mode("overwrite").save(external_path)
spark.sql(f"""create table employee_external using delta location '{external_path}'""")

In [0]:
spark.sql("DESCRIBE DETAIL employee_external").display()

In [0]:
new_data = [(5,"David",50000),
        (6,"shiva",70000),
        (7,"kiran",9000)]
df_new = spark.createDataFrame(new_data,columns)
df_new.display()  

In [0]:
df_new.write.format("delta").mode("append").saveAsTable("employee_managed")

In [0]:
%sql
select * from employee_managed

In [0]:
%sql
update employee_managed set salary = 100000 where name = 'Sana';

In [0]:
delta_emp = DeltaTable.forName(spark, "employee_managed")
delta_emp.update(
    condition = "name='Sana'",
    set = {"salary": "12000"}
)

In [0]:
%sql
select * from employee_managed

sql merge for upsert/update

In [0]:
%sql
MERGE INTO employee_managed target
USING (
  SELECT 2 AS id, 'Sana' AS name, 90000 AS salary
  UNION ALL
  SELECT 6 AS id, 'Ravi' AS name, 10000 AS salary
) AS source
ON target.id = source.id
WHEN MATCHED THEN 
  UPDATE SET 
    target.name = source.name,
    target.salary = source.salary
WHEN NOT MATCHED THEN 
  INSERT (id, name, salary)
  VALUES (source.id, source.name, source.salary);


In [0]:
%sql
select * from employee_managed

In [0]:
%sql
DESCRIBE HISTORY employee_managed;

In [0]:
df3v=spark.read.format("delta").option("versionAsOf", 2).table("employee_managed")
df3v.show()

In [0]:
%sql
VACUUM employee_managed RETAIN 168 HOURS;


In [0]:
%sql
DESCRIBE HISTORY employee_managed;

In [0]:
%sql
SELECT * FROM employee_managed VERSION AS OF 8; -- works if within 7 days
